## Neural Network

In [1]:
import numpy as np
import matplotlib.pyplot as mpl
from scipy import optimize

In [2]:
X = np.loadtxt('xy/x.csv', delimiter=',')
Y = np.loadtxt('xy/y.csv', delimiter=',')
T1 = np.loadtxt('theta/theta1.csv', delimiter=',')
T2 = np.loadtxt('theta/theta2.csv', delimiter=',')

In [3]:
def sigmoid(a):
    return 1.0 / (1 + np.exp(-a))

In [4]:
def porcent(res, y):
    contador = 0
    for i in range(res.shape[0]) :
        if ((res[i]) == Y[i]):
            contador +=1
    
    return (contador*100.0)/res.shape[0]

In [5]:
def init_weight(input_layer, hidden_layer, out_layer):
    np.random.seed(2) 
    w1 = np.random.randn(hidden_layer, input_layer +1) 
    w2 = np.random.randn(out_layer, hidden_layer +1)
    return w1,w2

In [6]:
#vectorizando Y
def create_Y(y, out_size, m):
    yn = np.zeros((m, out_size))
    for i in range(m):
        yn[i][int(y[i])%10] = 1
    return yn

In [32]:
def matrix_conf(y_, y_pre, val):
    matrix = np.zeros((val,val))
    for i in range(y_pre.shape[0]):
        k = int(y_[i]-1) #valores reales
        l = int(y_pre[i]-1) #valores predichos
        matrix[l][k] += 1
    return matrix

In [8]:
def compute_cost(a3, Y):
    m = Y.shape[0] # 5000
    temp = np.sum(Y * np.log(a3) + (1 - Y) * (np.log(1 - a3)), axis=1)
    cost = (-1/m) * np.sum(temp, axis=0)
    return cost

In [9]:
def gradiente_prima(z):
    gz = sigmoid(z)
    return gz * (1-gz);

### Forward

In [10]:
def forward(x, t1, t2):
    #capa1
    a1 = np.c_[np.ones((x.shape[0],1)),x] 
    #capa2
    z2 = np.dot(a1, t1.T)
    a2= sigmoid(z2)
    a2 = np.c_[np.ones((z2.shape[0],1)),a2] 
    #capa3
    z3 = np.dot(a2,t2.T) 
    a3 = sigmoid(z3)
    
    result={'a1': a1, 'a2':a2, 'a3':a3, 'z2':z2, 'z3':z3}    
    return result

### Compute Gradiente

In [11]:
def compute_gradient_reg(ford,t2, y_vec , m):
    a1 = ford['a1']
    a2 = ford['a2']
    a3 = ford['a3']
    z2 = ford['z2']
    z3 = ford['z3']
        
    d3 = a3 - y_vec 
    d2 = np.dot(d3,t2) * gradiente_prima(np.c_[np.ones((z2.shape[0],1)),z2])
    d2 = d2[:,1:] 
    
    delta1 = np.dot(d2.T,a1)
    delta2 = np.dot(d3.T,a2) 
    
    grad1 = delta1/m
    grad2 = delta2/m
        
    return grad1,grad2

In [17]:
def session(x, y, hidden_layer, iterations):
    input_layer = x.shape[1]
    out_layer = y.shape[1] 
    m = x.shape[0]
    rate = 1.5
    
    w1,w2 = init_weight(input_layer, hidden_layer, out_layer)
    
    for i in range(iterations):
        ford = forward(x, w1, w2)
        cost = compute_cost(ford['a3'], y) 
        grad1,grad2 = compute_gradient_reg(ford, w2, y,m) #backward
        
        w1 = w1 - rate * grad1
        w2 = w2 - rate * grad2
        
        if i % 100 == 0:
            print (i," iteracion con costo: ",cost)
    return w1,w2

In [34]:
Y_vec = create_Y(Y, 10, X.shape[0])
W1,W2 = session(X, Y_vec, 25,3001)

0  iteracion con costo:  15.798839856850712
100  iteracion con costo:  1.234297050087826
200  iteracion con costo:  0.9023178323822167
300  iteracion con costo:  0.7398117518912719
400  iteracion con costo:  0.6411855886371334
500  iteracion con costo:  0.5713903206841736
600  iteracion con costo:  0.517753563956345
700  iteracion con costo:  0.4742257849468672
800  iteracion con costo:  0.436798165068693
900  iteracion con costo:  0.40418059375835047
1000  iteracion con costo:  0.3761748598695956
1100  iteracion con costo:  0.35200162503246685
1200  iteracion con costo:  0.3310177717388724
1300  iteracion con costo:  0.3125809067255506
1400  iteracion con costo:  0.29618860000432473
1500  iteracion con costo:  0.2814753916837179
1600  iteracion con costo:  0.2682056725283301
1700  iteracion con costo:  0.2561762040865273
1800  iteracion con costo:  0.24516861327712414
1900  iteracion con costo:  0.23499559026972858
2000  iteracion con costo:  0.2255423397890966
2100  iteracion con cos

In [20]:
# Mi prediccion
ford = forward(X, W1, W2)
out = ford['a3']
out

array([[9.99542396e-01, 6.38537810e-06, 6.07722307e-05, ...,
        4.19016589e-03, 2.46241833e-05, 8.04992846e-04],
       [9.99601498e-01, 4.86269175e-06, 6.34309020e-05, ...,
        6.44818303e-03, 2.82201151e-05, 4.92758673e-04],
       [9.58805015e-01, 2.17251776e-05, 1.04313607e-04, ...,
        2.83603010e-05, 1.01358620e-02, 9.29428240e-02],
       ...,
       [3.45241760e-07, 2.03140450e-01, 7.94978421e-05, ...,
        2.90930624e-07, 1.06762217e-02, 6.61405322e-01],
       [2.51054489e-07, 2.72335699e-04, 2.34584195e-08, ...,
        2.83776383e-03, 3.17581239e-03, 8.70985313e-01],
       [2.53486070e-02, 1.24279490e-05, 1.24018181e-02, ...,
        2.27665996e-01, 6.46383062e-05, 7.47431624e-01]])

In [33]:
result = np.argmax(out,axis=1).T #prediccion  
matrix_conf(Y, result, 10)

array([[494.,   1.,   0.,   1.,   0.,   1.,   2.,   0.,   1.,   0.],
       [  1., 486.,   3.,   0.,   1.,   0.,   3.,   3.,   0.,   0.],
       [  1.,   1., 480.,   0.,   8.,   0.,   1.,   5.,   2.,   1.],
       [  0.,   2.,   0., 489.,   1.,   0.,   3.,   2.,   5.,   1.],
       [  2.,   0.,   8.,   1., 478.,   2.,   0.,   4.,   1.,   0.],
       [  0.,   3.,   1.,   2.,   1., 494.,   0.,   1.,   0.,   1.],
       [  0.,   3.,   1.,   0.,   0.,   0., 488.,   1.,   3.,   1.],
       [  2.,   2.,   3.,   0.,   7.,   1.,   2., 483.,   5.,   2.],
       [  0.,   0.,   3.,   7.,   2.,   0.,   1.,   1., 479.,   0.],
       [  0.,   2.,   1.,   0.,   2.,   2.,   0.,   0.,   4., 494.]])

In [22]:
porcent(result,Y)

87.42

### Precision y Recall

In [ ]:
#presicion